In [2]:
import numpy as np  # для работы с массивами и математическими функциями
import pandas as pd  # для обработки и анализа данных, особенно табличных данных
import matplotlib.pyplot as plt  # для создания и настройки графиков
import seaborn as sns  # для стилизации и улучшенной визуализации графиков
import os
import glob

import optuna  # для оптимизации гиперпараметров модели с помощью продвинутых методов поиска
import lightgbm as lgb  # для использования алгоритма градиентного бустинга LightGBM
from sklearn.model_selection import train_test_split  # для разделения данных на обучающую и тестовую выборки
from sklearn.preprocessing import StandardScaler  # для нормализации данных перед обучением
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # для оценки качества модели
from lightgbm import early_stopping  # для реализации ранней остановки в процессе обучения LightGBM
from sklearn.metrics import roc_auc_score

In [3]:
df_hack_test = pd.read_csv('C:\\Users\\ilyak\\Desktop\\Python\\Python_projects\\Материалы\\Test_датасет_альфа')
df_hack_test.head()

,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,test,38731,0.262983,-1.153775,1.0,-0.648932,-0.847237,-0.462393,-1.215373,0.605520,...,-0.236218,-1.058259,0.0,0.835147,-0.176562,-1.455333,-0.766280,-1.365915,0.454763,0.809724
1,test,38732,-0.698407,-1.529096,1.0,0.033011,-1.166594,-0.889886,-0.900391,-1.201925,...,-1.157688,0.334938,0.0,-0.231860,-0.080340,-0.493846,-1.245681,-1.632521,0.246708,-0.226893
2,test,38733,-0.331403,-0.538437,1.0,-1.531462,-0.924031,-1.062342,-1.789274,-1.097902,...,-1.159555,0.351526,0.0,-1.375061,-0.156735,-0.437773,-1.851818,0.984694,0.630962,0.279446
3,test,38734,0.675162,-0.077064,1.0,-1.893065,0.610631,-1.404934,-2.650392,-1.752463,...,0.180918,0.269593,0.0,1.054666,-0.219652,-0.438285,-0.707442,-1.498935,0.962498,-0.390251
4,test,38735,-2.162385,-1.737901,1.0,-1.899384,-1.384151,0.106314,-0.814678,1.111808,...,0.696544,1.006378,0.0,1.107787,-0.282357,-0.235000,-1.794316,-0.504509,-0.385182,-1.130777


In [4]:
df_hack_train = pd.read_csv('C:\\Users\\ilyak\\Desktop\\Python\\Python_projects\\Материалы\\Train_датасет_альфа')
df_hack_train.head()

,target,smpl,id,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,...,feature_409,feature_410,feature_411,feature_412,feature_413,feature_414,feature_415,feature_416,feature_417,feature_418
0,0,train,0,0.468142,-1.045346,0.0,0.384487,0.435121,-1.178548,0.124543,...,-0.361507,-1.026853,0.0,1.418600,-0.929668,1.284014,0.731842,0.801786,-0.728297,-0.412095
1,0,train,1,-0.760983,0.515132,0.0,-1.673905,-0.393862,-1.584207,-0.439778,...,-0.546275,-1.489542,0.0,-0.622007,-0.473156,0.780020,0.648577,0.646100,-0.789362,0.083349
2,0,train,2,1.658855,0.915052,0.0,-0.581082,0.477199,-0.622226,0.390642,...,-0.485999,0.586012,0.0,0.361481,-0.364566,-1.318596,-0.385155,0.140133,0.123245,-0.670030
3,0,train,3,-0.638854,0.314099,0.0,0.000919,1.102342,-0.807371,0.329158,...,0.321985,-0.075827,0.0,-1.629672,0.876864,0.411271,0.433440,0.997364,2.829590,-1.275588
4,0,train,4,-1.091376,0.859811,0.0,-0.505439,1.665086,-0.912464,-0.332054,...,0.828886,0.140387,0.0,-0.624304,-2.197691,-1.479267,-0.465917,-0.014757,-0.320434,-0.511896


In [5]:
X = df_hack_train.drop(['target','smpl', 'id'], axis=1)
y = df_hack_train['target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=101)

In [13]:
# Функция для настройки и обучения модели с использованием Optuna
from sklearn.model_selection import StratifiedKFold
def objective(trial):
    # Определение гиперпараметров для LightGBM с использованием Optuna
    param = {
        "objective": "binary",  # Бинарная классификация (предсказание вероятности возврата кредита)
        "metric": "auc",  # Метрика качества для оптимизации - AUC (площадь под ROC-кривой)
        "boosting_type": "dart ",  # Использование градиентного бустинга
        "device": "gpu",  # Указание использования GPU для ускорения обучения
        "verbosity": -1,  # Отключение логирования LightGBM для уменьшения вывода на экран

        # Параметр регуляризации L1 (Lasso) для предотвращения переобучения
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),

        # Параметр регуляризации L2 (Ridge) для предотвращения переобучения
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),

        # Количество листьев в одном дереве; большее значение позволяет лучше подстраиваться под данные
        "num_leaves": trial.suggest_int("num_leaves", 2, 2048),

        # Доля случайно отобранных признаков для каждого дерева (Feature Sampling)
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.0001, 1.0),

        # Доля данных для обучения каждого дерева (Bagging Sampling)
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.0001, 1.0),

        # Частота выполнения bagging, т.е. каждая N-я итерация
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 9),

        # Минимальное количество объектов в каждом листе для контроля глубины деревьев и предотвращения переобучения
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 25),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.001, 10.0)
    }

    # Создание датасетов для LightGBM
# dtrain - обучающий набор данных, содержащий признаки и целевые метки
# dvalid - валидационный набор данных, используется для контроля переобучения и ранней остановки
# reference=dtrain - указывает, что dvalid является ссылочным датасетом для dtrain, что позволяет модели учитывать структуру dtrain при оптимизации
    dtrain = lgb.Dataset(X_train, label=y_train)
    dvalid = lgb.Dataset(X_valid, label=y_valid, reference=dtrain)

# Обучение модели с использованием ранее определённых гиперпараметров
# num_boost_round=5000 - максимальное количество итераций обучения (число деревьев)
# early_stopping_rounds=100 - ранняя остановка, если модель не улучшает метрику AUC на валидационных данных в течение 100 итераций
# valid_sets=[dvalid] - контроль качества модели на валидационных данных
# verbose_eval=False - отключение логирования на каждой итерации
    model = lgb.train(
        param, dtrain, valid_sets=[dvalid],
        num_boost_round=10000,
        callbacks=[# Колбэки для управления процессом обучения
            lgb.early_stopping(stopping_rounds=1000),  # Остановка обучения, если метрика не улучшается в течение 300 итераций
            lgb.log_evaluation(period=1000)  # Логирование метрики каждые 100 итераций
            ]
        )

# Предсказание на валидационном наборе данных
# model.predict(X_valid) возвращает предсказанные вероятности классов для каждого объекта
    preds = model.predict(X_valid)

# Оценка модели с помощью метрики AUC (площадь под ROC-кривой)
# roc_auc_score вычисляет AUC, что позволяет измерить качество классификации (1 - идеально, 0.5 - случайно)
    auc = roc_auc_score(y_valid, preds)


    return auc


In [14]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [15]:
import time

# Сохранение времени начала для первой итерации
start_time = time.time()

def print_progress(study, trial):
    global start_time
    # Проверка, является ли номер испытания кратным 5
    if trial.number % 10 == 0:
        # Вычисление времени, затраченного с момента предыдущей 5-й итерации (или начала)
        elapsed_time = time.time() - start_time
        start_time = time.time()  # Обновляем стартовое время для следующей итерации

        print(f"Trial {trial.number} finished with value: {trial.value} and parameters: {trial.params}")
        print(f"Time elapsed for last 10 trials: {elapsed_time:.2f} seconds")
        print(f"Best trial so far: Trial {study.best_trial.number} with value: {study.best_trial.value}")

optuna.logging.set_verbosity(optuna.logging.WARNING)

In [16]:
# Запуск мониторинга GPU перед началом выполнения других задач (например, обучения модели)
# Функция будет выводить информацию о состоянии GPU каждые 60 секунд
#monitor_gpu(interval=60)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1000, callbacks=[print_progress],n_jobs=-1)

Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
Training until validation scores don't improve for 1000 rounds
[1000]	valid_0's auc: 0.5
Early stopping, best iteration is:
[1]	valid_0's auc: 0.5
Trial 10 finished with value: 0.5 and parameters: {'lambda_l1': 7.882264991406024e-08, 'lambda_l2': 0.00015397637706843806, 'num_leaves': 1276, 'feature_fractio

In [17]:
# Получение наилучших параметров
best_params = study.best_params
print("Лучшие параметры:", best_params)

# Обучение финальной модели с оптимизированными параметрами
final_model = lgb.train(
    {**best_params, "objective": "binary", "metric": "auc", "boosting_type": "gbdt", "device": "gpu"},
    lgb.Dataset(X_train, label=y_train),
    num_boost_round=10000,
    valid_sets=[lgb.Dataset(X_valid, label=y_valid)]
)

Лучшие параметры: {'lambda_l1': 1.7357456442284624e-05, 'lambda_l2': 0.41357402165388285, 'num_leaves': 1212, 'feature_fraction': 0.8813284858641731, 'bagging_fraction': 0.8516091214219346, 'bagging_freq': 9, 'min_child_samples': 766, 'max_depth': 3, 'scale_pos_weight': 1.2543497570303155}
[LightGBM] [Info] Number of positive: 584, number of negative: 26527
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 97310
[LightGBM] [Info] Number of data points in the train set: 27111, number of used features: 418
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GTX 1650, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 393 dense feature groups (10.24 MB) transferred to GPU in 0.015355 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.021541 -> initscore=-3.816017
[LightGBM] [Info] Start train

In [20]:
# Оценка финальной модели на тестовой выборке
final_preds = final_model.predict(X_valid)
final_auc = roc_auc_score(y_valid, final_preds)
print("AUC на тестовой выборке:", final_auc)

AUC на тестовой выборке: 0.692059028737138


In [21]:
import joblib

# Сохранение модели в формате .pkl с использованием joblib
joblib.dump(final_model, "Test_Alpha_Hack_LightGBM_Optuna1.pl")


['Test_Alpha_Hack_LightGBM_Optuna1.pl']

In [ ]:
# Загрузка модели из файла
import joblib
pretrained_lgb_model = joblib.load("Test_Alpha_Hack_LightGBM_Optuna1.pl")